# Results:

[Functional Annotation of the genes associated with the top 5 _environment_ SNPs](#Functional-Annotation-of-the-genes-associated-with-the-top-5-environment-SNPs:)

[Functional Annotation of the genes associated with the top 5 _infection_ SNPs](#Functional-Annotation-of-the-genes-associated-with-the-top-5-infection-SNPs:)

# Purpose:

2014-12-26 (Friday)

Create code to make tables with info from `argot2` annotations for all genes in a gene-list.

# Implementation:

## Imports:

In [79]:
# imports
from collections import defaultdict

import pandas as pd

import munch

from IPython.display import display, HTML

from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes
from spartan.utils.files import tableFile2namedTuple

## File paths:

In [2]:
# define paths to files
bpth = "/home/gus/remote_mounts/louise/data/"

## basic genome files
fanno = bpth + "genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

## project specific files
top1_environment = bpth + "projects/ddrad58/SNPs_of_interest/genes_near_SNPs/tsetseFINAL_14Oct2014_f2_53_v_GfusI1.1.window.5000.top1_env.tsv"
top1_infection = bpth + "projects/ddrad58/SNPs_of_interest/genes_near_SNPs/tsetseFINAL_14Oct2014_f2_53_v_GfusI1.1.window.5000.top1_infection.tsv"

top5_environment = bpth + "projects/ddrad58/SNPs_of_interest/genes_near_SNPs/tsetseFINAL_14Oct2014_f2_53_v_GfusI1.1.window.5000.top5_env.tsv"
top5_infection = bpth + "projects/ddrad58/SNPs_of_interest/genes_near_SNPs/tsetseFINAL_14Oct2014_f2_53_v_GfusI1.1.window.5000.top5_infection.tsv"



## Set up main data variables:

### Set up annotation database:

In [3]:
argot2 = pd.read_hdf(path_or_buf=fanno, key='dataframe')

In [4]:
argot2.head()

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
0,GFUI034947-PA,P,GO:0006508,proteolysis,270.313447,0.496543,8.247696
1,GFUI035874-PA,F,GO:0005515,protein binding,529.038456,0.500000,5.471582
2,GFUI033625-PA,P,GO:0005980,glycogen catabolic process,307.758251,0.284597,13.424450
3,GFUI033625-PA,F,GO:0004134,4-alpha-glucanotransferase activity,159.513252,0.177063,10.153643
4,GFUI033625-PA,F,GO:0004135,"amylo-alpha-1,6-glucosidase activity",184.352303,0.177063,11.734746


### Set up gene/SNP relationship data:

#### Function to create dictionary-based retrieval object for gene/SNP data:

In [33]:
def bed3_v_gff3_window_by_gff3_ID(bedtools_window_out):
    """
    Returns dictionary-based retrieval object for Bedtools result from 3-field BED format records vs
    GFF3 format records data grouped by the GFF3 features.
    """
    headers = ["bed3_seq",
               "bed3_start",
               "bed3_end",
               "gff3_seq",
               "gff3_source",
               "gff3_type",
               "gff3_start",
               "gff3_end",
               "gff3_score",
               "gff3_strand",
               "gff3_phase",
               "gff3_attributes",]
    
    table = tableFile2namedTuple(bedtools_window_out, 
                                 sep='\t', 
                                 headers=headers)
    
    Tree = lambda: defaultdict(Tree)
    data = Tree()
    
    for row in table:
        gff3_rec = parse_gff3_attributes(row.gff3_attributes)['ID']

        data[gff3_rec]['info'].setdefault('seq', row.gff3_seq)
        data[gff3_rec]['info'].setdefault('source', row.gff3_source)
        data[gff3_rec]['info'].setdefault('type', row.gff3_type)
        data[gff3_rec]['info'].setdefault('start', row.gff3_start)
        data[gff3_rec]['info'].setdefault('end', row.gff3_end)
        data[gff3_rec]['info'].setdefault('score', row.gff3_score)
        data[gff3_rec]['info'].setdefault('strand', row.gff3_strand)
        data[gff3_rec]['info'].setdefault('phase', row.gff3_phase)
        data[gff3_rec]['info'].setdefault('attributes', row.gff3_attributes)

        bed3_hit = dict(seq = row.bed3_seq, 
                        start = row.bed3_start,
                        end = row.bed3_end)

        try:
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

        except AttributeError:
            data[gff3_rec]['bed3_hits'] = []
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

    return munch.munchify(data)

In [51]:
top5_environment_search = bed3_v_gff3_window_by_gff3_ID(top5_environment)

In [35]:
len(top5_environment_search)

83

In [36]:
top5_environment_search.keys()[:3]

['GFUI003362', 'GFUI017236', 'GFUI022267']

In [37]:
top5_environment_search['GFUI003362']['bed3_hits']

[Munch(end='164021', seq='Scaffold109', start='164020')]

In [43]:
top5_environment_search['GFUI003362'].bed3_hits

[Munch(end='164021', seq='Scaffold109', start='164020')]

## Create tables for our genes that reference the SNP location they were associated with:

In [47]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for index, row in enumerate(self):
            html.append("<tr>")
            
            for col in row:
                # if header, format as such
                if index == 0:
                    html.append("<th>{0}</th>".format(col))
                else:
                    html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [52]:
# load SNP search data
top1_environment = bed3_v_gff3_window_by_gff3_ID(top1_environment)
top1_infection   = bed3_v_gff3_window_by_gff3_ID(top1_infection)

top5_environment = bed3_v_gff3_window_by_gff3_ID(top5_environment)
top5_infection   = bed3_v_gff3_window_by_gff3_ID(top5_infection)

In [87]:
top5_environment.GFUI003362.bed3_hits

[Munch(end='164021', seq='Scaffold109', start='164020')]

In [55]:
def filter_by_gene(gene, argot_df):
    return argot_df[argot_df.Sequence.str.startswith(gene)]

In [59]:
def sort_by_TS(argot_df):
    return argot_df.sort('Total Score', ascending=0)

In [61]:
def filter_by_TS(tscore, argot_df):
    return argot_df[argot_df["Total Score"] >= tscore]

In [81]:
def gene_table(gene_name, argot_df, tscore):
    return sort_by_TS(filter_by_TS(tscore, filter_by_gene(gene_name, argot_df)))

In [74]:
def vb_link(gene_name, link_template):
    return HTML(link_template % dict(gene_name=gene_name))

In [89]:
def format_snps(snp_list):
    snp_line = "%(seq)s:%(end)s"
    
    snp_strs = []
    
    for snp in snp_list:
        snp_strs.append(snp_line % snp)
    
    br = "<br>"
    return br.join(snp_strs) + br

In [83]:
# Vectorbase Link Templates
gene_page          = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Gene/Summary?db=core;g=%(gene_name)s"> %(gene_name)s: gene home page</a>'''
protein_summary    = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/ProteinSummary?db=core;g=%(gene_name)s"> %(gene_name)s: protein summary </a>'''
gene_ontology      = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Gene/Ontology/molecular_function?db=core;g=%(gene_name)s;oid=molecular_function"> %(gene_name)s: gene ontology </a>'''
gene_region_detail = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Location/View?db=core;g=%(gene_name)s;r"> %(gene_name)s: genome browser </a>'''

In [93]:
def present_gene(gene_name, nearby_snps):
    display(HTML("<h2>%s</h2>" % (gene_name)))
    
    nearby_snps = format_snps(nearby_snps)
    display(HTML("<b>SNP location:</b><br>%s" % (nearby_snps)))
    
    display(gene_table(gene_name=gene_name,
                       argot_df=argot2,
                       tscore=200))
    
    display(HTML("<p>"))
    display(vb_link(gene_name=gene_name, link_template=gene_page))
    display(vb_link(gene_name=gene_name, link_template=protein_summary))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology))
    display(vb_link(gene_name=gene_name, link_template=gene_region_detail))
    display(HTML("<hr>"))

# Functional Annotation of the genes associated with the top 5 environment SNPs:

In [94]:
for gene in top5_environment:
    
    snps = top5_environment[gene].bed3_hits
    present_gene(gene_name=gene, nearby_snps=snps)
    

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


...

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
27734,GFUI027255-PA,F,GO:0008199,ferric iron binding,3520.667148,0.294476,8.659780
27735,GFUI027255-PA,C,GO:0005739,mitochondrion,1119.291606,0.695381,5.085303


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
16830,GFUI041872-PA,F,GO:0004896,cytokine receptor activity,259.772571,0.563888,10.488532


# Functional Annotation of the genes associated with the top 5 infection SNPs:

In [95]:
for gene in top5_infection:
    
    snps = top5_infection[gene].bed3_hits
    present_gene(gene_name=gene, nearby_snps=snps)

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
62449,GFUI026694-PA,P,GO:0019307,mannose biosynthetic process,1986.500391,0.185032,15.678973
62453,GFUI026694-PA,C,GO:0005737,cytoplasm,1876.313784,0.773891,3.178476
62451,GFUI026694-PA,F,GO:0004615,phosphomannomutase activity,1509.276050,0.287575,11.148084
62452,GFUI026694-PA,F,GO:0016853,isomerase activity,511.394510,0.545867,4.815628
62450,GFUI026694-PA,F,GO:0003824,catalytic activity,272.359017,0.931151,1.410349


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
65443,GFUI026692-PA,P,GO:0019307,mannose biosynthetic process,2775.627555,0.202663,15.678973
65447,GFUI026692-PA,C,GO:0005737,cytoplasm,2039.739454,0.704821,3.178476
65445,GFUI026692-PA,F,GO:0004615,phosphomannomutase activity,1879.359140,0.298882,11.148084
65446,GFUI026692-PA,F,GO:0016853,isomerase activity,608.639688,0.561327,4.815628
65444,GFUI026692-PA,F,GO:0003824,catalytic activity,397.605948,0.937166,1.410349


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
20316,GFUI051764-PA,F,GO:0003998,acylphosphatase activity,4648.556871,0.426202,10.560419
20317,GFUI051764-PA,F,GO:0016787,hydrolase activity,764.849275,0.753257,2.643033
20319,GFUI051764-PA,C,GO:0070062,extracellular vesicular exosome,537.389311,0.331108,8.572522
20315,GFUI051764-PA,P,GO:0008152,metabolic process,363.072060,0.908530,0.544703
20318,GFUI051764-PA,C,GO:0005739,mitochondrion,260.556240,0.404137,5.085303


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


...

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
54024,GFUI012091-PA,C,GO:0005634,nucleus,4058.040375,0.845448,5.725783
54017,GFUI012091-PA,P,GO:0006351,"transcription, DNA-templated",1863.974037,0.549085,6.856772
54020,GFUI012091-PA,F,GO:0003899,DNA-directed RNA polymerase activity,1849.001878,0.226232,7.248868
54019,GFUI012091-PA,F,GO:0008270,zinc ion binding,905.748570,0.163055,5.400313
54022,GFUI012091-PA,F,GO:0003677,DNA binding,539.783982,0.157716,3.831010
54021,GFUI012091-PA,F,GO:0003676,nucleic acid binding,529.591939,0.334679,3.285748
54023,GFUI012091-PA,C,GO:0005666,DNA-directed RNA polymerase III complex,419.333384,0.148226,12.676194
54018,GFUI012091-PA,F,GO:0046872,metal ion binding,284.586728,0.310144,3.508837


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
56191,GFUI011599-PA,F,GO:0030246,carbohydrate binding,832.391238,0.417392,6.799836
56192,GFUI011599-PA,C,GO:0016020,membrane,782.918094,0.491665,2.472824
